In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow import math

from sklearn.metrics import roc_curve
from utils import load_dataset
import matplotlib.pyplot as plt
import numpy as np

In [7]:
from tensorflow.config import list_physical_devices

list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
def schedule(epoch, lr) -> float:
    if epoch >= 200 and epoch % 25 == 0:
        lr = lr * math.exp(-0.1)
        print(f'Epoch {epoch}\tNew LR: {lr}')
    return lr

In [ ]:
scheduler = LearningRateScheduler(schedule)
es = EarlyStopping(monitor='loss', patience=25)
optimizer = Adam(lr=1e-3)

epochs = 500
validation_freq = 5
boundary = 300

In [ ]:
for n in [1, 2, 3, 5, 8]:
    print(f'--------------- Starting model {n} ---------------')
    X_train, y_train, X_test, y_test = load_dataset(f'm{n}')

    clear_session()
    model = Sequential([
                        LSTM(4, input_shape=(4, 1000)),
                        Dense(8, activation='relu'),
                        Dense(1, activation='relu')
    ])
    model.compile(optimizer=optimizer, loss='mse')

    history = model.fit(X_train,
                        y_train,
                        epochs=epochs,
                        callbacks=[scheduler, es],
                        validation_data=(X_test, y_test),
                        validation_freq=validation_freq,
                        verbose=1
    )

    passed_epochs = len(history.history['loss'])
    plt.plot(range(passed_epochs), history.history['loss'], label='loss')
    plt.plot(range(validation_freq, passed_epochs + 1, validation_freq),
             history.history['val_loss'],
             label='val loss'
    )
    plt.legend()
    plt.show()

    y = np.where(y_test > boundary, 0, 1)
    pred = np.where(model.predict(X_test) > boundary, 0, 1)

    fpr, tpr, thresholds = roc_curve(y, pred)

    plt.title('ROC curve')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.plot(tpr, fpr)
    plt.show()

    print(f'Thresholds: {thresholds}')
    model.save(f'lstm-{n}-model.h5')